# A Step-by-Step Guide to Holoclean example

Noisy and erroneous data is a major bottleneck in analytics. Data cleaning and repairing account for about 60% of the work of data scientists. To address this bottleneck, we recently introduced HoloClean, a semi-automated data repairing framework that relies on statistical learning and inference to repair errors in structured data. In HoloClean, we build upon the paradigm of weak supervision and demonstrate how to leverage diverse signals, including user-defined heuristic rules (such as generalized data integrity constraints) and external dictionaries, to repair erroneous data. 

In this post, we walk through the process of implementing Holoclean, by creating a simple end-to-end example.

# Setup

Firstly, we import all the module from Holoclean that we will use.

In [1]:
from holoclean.holoclean import HoloClean, Session
from holoclean.errordetection.errordetector import ErrorDetectors
from holoclean.featurization.featurizer import SignalInit, SignalCooccur, SignalDC
from holoclean.featurization.featurizer import Featurizer
from holoclean.learning.softmax import SoftMax
from holoclean.learning.accuracy import Accuracy
import time

##   Initialization
In this part, we create the Holoclean and Session object that we will use for this example.

In [2]:
holo_obj = HoloClean()
session = Session("Session", holo_obj)
        

/home/james/anaconda/envs/python2.7/lib/python2.7/site-packages/sqlalchemy/dialects/mysql/base.py:1576: Warning: '@@tx_isolation' is deprecated and will be removed in a future release. Please use '@@transaction_isolation' instead
  cursor.execute('SELECT @@tx_isolation')


## Read Input and DC from file
Test data and the Denial Constraints will be read using the Session's ingestor.
After ingesting the test data will be loaded into MySQL tables along with entries in the a metadata table.

In [3]:
fx = open('execution_time.txt', 'w')
# list_time = []
# start_time = t()
t0 = time.time()
#session.ingest_dataset("test/inputDatabase.csv")
session.ingest_dataset("test/test.csv")
# session.ingest_dataset("test/test1.csv")

t1 = time.time()

total = t1 - t0
fx.write('time for ingesting file: ' + str(total) + '\n')
print 'time for ingesting file: ' + str(total) + '\n'
#session.denial_constraints("test/inputConstraint.txt")
session.denial_constraints("test/dc1.txt")
# session.denial_constraints("test/dc2.txt")

print 'Init table'
sql = holo_obj.dataengine.get_table_to_dataframe("Init", session.dataset)
#sql.select('index','ProviderNumber','HospitalName', 'Address1').show()

sql.show()

time for ingesting file: 9.41983699799

Init table
+-----+---+---+---+---+---+
|index|  A|  B|  C|  D|  E|
+-----+---+---+---+---+---+
|    1|  p|  w|  f|  n|  r|
|    2|  p|  z|  f|  k|  r|
|    3|  u|  y|  m|  n|  r|
+-----+---+---+---+---+---+



## Error Detection
In this part, we create the error detection. The output of this part is the C_dk table that contains all the noisy cells and the C_Clean table that contains the clean cells

In [6]:
 t0 = time.time()
err_detector = ErrorDetectors(session.Denial_constraints, holo_obj.dataengine,
                             holo_obj.spark_session, session.dataset)
session.add_error_detector(err_detector)
session.ds_detect_errors()

t1 = time.time()
total = t1 - t0
holo_obj.logger.info('error dectection time: '+str(total)+'\n')
fx.write('error dectection time: '+str(total)+'\n')
print 'error dectection time: '+str(total)+'\n'

SELECT table1.index as ind,table2.index as                indexT2 FROM df table1,df table2 WHERE (table1.A=table2.A AND table1.B<>table2.B)
SELECT table1.index as ind,table2.index as                indexT2 FROM df table1,df table2 WHERE (table1.C=table2.C AND table1.D<>table2.D)
error dectection time: 45.7984521389



## Domain Pruning
In this part, we prune the domain. The output of this part is the possible_values tables that contains all the possible values for each cell

In [7]:
t0 = time.time()
pruning_threshold = 0.5
session.ds_domain_pruning(pruning_threshold)

t1 = time.time()
total = t1 - t0
holo_obj.logger.info('domain pruning time: '+str(total)+'\n')
fx.write('domain pruning time: '+str(total)+'\n')
print 'domain pruning time: '+str(total)+'\n'

print 'Possible_values_dk'
sql = holo_obj.dataengine.get_table_to_dataframe("Possible_values_clean", session.dataset)
sql.show()

print 'Possible values dk'
sql = holo_obj.dataengine.get_table_to_dataframe("Possible_values_dk", session.dataset)
sql.show()

domain pruning time: 4.60110998154

Possible_values_dk
+---+---+---------+--------+--------+---------+
|vid|tid|attr_name|attr_val|observed|domain_id|
+---+---+---------+--------+--------+---------+
|  1|  3|        A|       p|       0|        1|
|  3|  3|        C|       m|       1|        1|
|  2|  3|        B|       y|       1|        1|
|  3|  3|        C|       f|       0|        2|
|  4|  3|        D|       n|       1|        1|
|  1|  3|        A|       u|       1|        2|
+---+---+---------+--------+--------+---------+

Possible values dk
+---+---+---------+--------+--------+---------+
|vid|tid|attr_name|attr_val|observed|domain_id|
+---+---+---------+--------+--------+---------+
|  6|  2|        B|       z|       1|        1|
|  4|  1|        D|       n|       1|        1|
|  1|  1|        A|       p|       1|        1|
|  5|  2|        A|       p|       1|        1|
|  7|  2|        C|       f|       1|        1|
|  2|  1|        B|       w|       1|        1|
|  8|  2|    

# Featurization

In this part, we implement the featurization module of holoclean. We choose the signals that we want to use and the output of this part is the featurization table that contains the factors that we will use

## Feature Signals

In [8]:
t0 = time.time()
initial_value_signal = SignalInit(session.Denial_constraints, holo_obj.dataengine,
                              session.dataset)
session.add_featurizer(initial_value_signal )
statistics_signal = SignalCooccur(session.Denial_constraints, holo_obj.dataengine,
                              session.dataset )
session.add_featurizer(statistics_signal)
dc_signal = SignalDC(session.Denial_constraints, holo_obj.dataengine, session.dataset,
                 holo_obj.spark_session)
session.add_featurizer(dc_signal)
t1 = time.time()
total = t1 - t0
print "Feature Signal Time:", total

Feature Signal Time: 0.000380992889404


We use the signals that we choose in the previous step. The output of this part is the featurization table that contains the factors that we will use in the next step.

In [9]:
t0 = time.time()
session.ds_featurize()

t1 = time.time()

total = t1 - t0

holo_obj.logger.info('featurization time: '+str(total)+'\n')
fx.write('featurization time: '+str(total)+'\n')
print 'featurization time: '+str(total)+'\n'

print 'Feature table clean'
sql = holo_obj.dataengine.get_table_to_dataframe("Feature_clean", session.dataset)
sql.show()


Setting up Feature Threads
adding a 0.312334060669 
Creating parallel queries
SignalInit
done adding  SignalInit   0.00309801101685
adding a  SignalCooccur
0.0202610492706
Starting threads
Thread-7  Query Started 
Thread-7  Query Execution time:  0.880417823792
done adding  SignalCooccur   Thread-81.23641395569 
 Query Started adding a 
 SignalDC
Thread-8  Query Execution time:  0.121952056885
done adding  SignalDC Thread-8  Thread-6  Query Started  0.949510097504
 Query Started Thread-9
Thread-7
  Query Started  
 Query Started 
Thread-8  Query Execution time:  0.00944209098816
Thread-9  Query Execution time:  0.00940608978271
Thread-7  Query Execution time:  0.0745539665222
Thread-6  Query Execution time:  0.099592924118Total Featurization Queries time Before Union: 

2.29448390007
 Select * from 606022807473_Thread_6_clean UNION Select * from 606022807473_Thread_7_clean UNION Select * from 606022807473_Thread_8_clean UNION Select * from 606022807473_Thread_9_clean 
Union Time: 
0.04

#  Learning
We create the X-tensor from the feature_clean table and run softmax on it

In [10]:
t0 = time.time()
soft = SoftMax(holo_obj.dataengine, session.dataset, holo_obj.spark_session)

print(soft.logreg())
t1 = time.time()
total = t1 - t0

fx.write('time for training model: '+str(total)+'\n')
print 'time for training model: '+str(total)+'\n'



 0.0000e+00  0.0000e+00
 0.0000e+00 -1.0000e+07
 0.0000e+00  0.0000e+00
 0.0000e+00 -1.0000e+07
[torch.FloatTensor of size 4x2]

Variable containing:
 0.0245  0.9755
 1.0000  0.0000
 0.9363  0.0637
 1.0000  0.0000
[torch.FloatTensor of size 4x2]

time for training model: 0.261505842209



In this part, we use the new weight, to learn the probabilities for each value for the cells


In [11]:
t0 = time.time()
session.ds_featurize(0)
t1 = time.time()
total = t1 - t0
fx.write('time for test featurization: ' + str(total) + '\n')
print 'time for test featurization: ' + str(total) + '\n'


print 'Feature table dk'
sql = holo_obj.dataengine.get_table_to_dataframe("Feature_dk", session.dataset)
sql.show()

t0 = time.time()
Y = soft.predict(soft.model, soft.setuptrainingX(), soft.setupMask(0))
print(Y)
t1 = time.time()
total = t1 - t0
print 'time for inference: ', total
soft.save_Y_to_db(Y)

print 'Inferred values for dk cells'
sql = holo_obj.dataengine.get_table_to_dataframe("Inferred_values", session.dataset)
sql.show()

Setting up Feature Threads
adding a 0.501136064529 SignalInit

Creating parallel queries
done adding  SignalInit   0.00367403030396
adding a  SignalCooccur
0.018905878067
Starting threads
Thread-12  Query Started 
Thread-12  Query Execution time:  0.912790060043
done adding  SignalCooccur   1.71454310417
adding a  SignalDC
Thread-13  Query Started 
Thread-13  Query Execution time:  0.138799905777
done adding  SignalDC   0.484138965607
Thread-14  Query Started Thread-13
  Query Started 
Thread-12  Query Started 
Thread-11  Query Started 
Thread-13  Query Execution time:  0.104687929153
Thread-14  Query Execution time:  Thread-110.138060092926 
 Query Execution time:  0.137963056564
Thread-12  Query Execution time:  0.238415002823
Total Featurization Queries time Before Union: 
2.44323992729
 Select * from 606022807473_Thread_11_dk UNION Select * from 606022807473_Thread_12_dk UNION Select * from 606022807473_Thread_13_dk UNION Select * from 606022807473_Thread_14_dk 
Union Time: 
0.1244